In [11]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import StaticAdjective, PointerAdjective, RankingAdjective
from src.explainer.explanation import AdjectiveExplanation, AssumptionExplanation, ConditionalExplanation, CompositeExplanation
from src.explainer.condition import Condition

In [12]:
# Example usage:
class MinMaxNode:
    def __init__(self, id, *, score=None, is_leaf, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.is_leaf = is_leaf
        self.children = children or []
        self.parent = None
        
        
        if len(self.children) > 0:
            for child in children:
                child.parent=self
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf1 = MinMaxNode('child1', score=3, is_leaf=True)
leaf2 = MinMaxNode('child2', score=5, is_leaf=True)
root = MinMaxNode('root', is_leaf=False, children=[leaf1, leaf2], score_child=leaf2)

In [13]:
framework = ArgumentationFramework()

framework.add_adjective(StaticAdjective("leaf", lambda node: node.is_leaf))

framework.add_adjective(PointerAdjective("score", 
    getter = lambda node: node.score,

    explanation = ConditionalExplanation(
        condition = Condition("leaf"),
        true_explanation = AssumptionExplanation("Leaf nodes have scores from the evaluation function"),
        false_explanation = CompositeExplanation(
            AssumptionExplanation("Internal nodes have scores from children"),
            AdjectiveExplanation("has score from child", "minoptimal")
))))

framework.add_adjective(RankingAdjective("better", "score", ">"))

# Renaming automatically created adjectives
framework.rename_adjective("max_better", "maxoptimal")
framework.rename_adjective("min_better", "minoptimal")

# Setting tree search motivation
framework.set_tree_search_motivation("maxoptimal")

framework.add_adjective(
    PointerAdjective("has score from child",
    #lambda node: min(node.children, key=lambda child: child.score),
    getter = lambda node: node.score_child,

    explanation = ConditionalExplanation(
        condition = Condition("minoptimal", "has score from child"),
        true_explanation = CompositeExplanation(
            AssumptionExplanation("We assume the opponent will do their best move."),
            AdjectiveExplanation("minoptimal", "has score from child")),
        false_explanation = AdjectiveExplanation("has score from child", "maxoptimal")
    )
))



In [14]:
explainer = ArgumentativeExplainer(framework)

Try

In [15]:
# Generate explanations
print(explainer.explain_adjective(root.children[0], "maxoptimal"))


((A node is maxranked in a RankingAdjective if the Ranking Condition is TRUE when compared to all its siblings.
 ∧ ((¬(child1.score > child2.score) → child1 not better than child2))) → child1 is not maxoptimal)


In [16]:
print(explainer.explain_adjective(root.children[0], "minoptimal"))

((A node is minranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings.
 ∧ ((¬(child1.score > child2.score) → child1 not better than child2))) → child1 is minoptimal)


In [17]:
print(explainer.explain_adjective(root.children[1], "maxoptimal"))

((A node is maxranked in a RankingAdjective if the Ranking Condition is TRUE when compared to all its siblings.
 ∧ ((child2.score > child1.score → child2 better than child1))) → child2 is maxoptimal)


In [18]:
print(explainer.explain_adjective(root, "score_child"))

KeyError: 'score_child'

In [ ]:
print(explainer.explain_adjective(root, "score"))

((¬(The node is is_leaf)
 ∧ (Internal nodes have scores from children
 ∧ ((A node is minranked in a RankingAdjective if the Ranking Condition is FALSE when compared to all its siblings.
 ∧ ((child2.score > child1.score → child2 better than child1))) → child2 is not minoptimal))) → root has score = 5)


In [ ]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

None
